<a href="https://colab.research.google.com/github/thejawker/architector/blob/master/Architector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Architector

> Algorithm to detect features of residential architecture.


## Possible Pipeline
1. Detect if image has house
2. Do a rough detection on labeled architecture
  - a. find a large dataset of house fronts
  - b. label these with the right architecture
  - c. train model
  - d. win
3. Recognise features (roof, window, shape of building, color, walls etc)
  - Either use supervised learning:
    - for these individual features have a huge amount of labeled data 
  - Or unsupervised
    - seperate the most distinct features in these

## Dataset
> How to get a good dataset?

#### Google Maps Street View

#### Scrape from Zillow, HotPads, Funda, etc.

### Labeling

## Things to test
Try running simple algorithm on lots of data and see if it picked up on good catagories through **unsupervised learning**.

## Roadmap

### Proof of concept
1. First do some testing, find 10 popular architecture styles
2. Find at least 100 images of those architecture styles
  - check pintrest
  - google images
  - zillow
  - hotpads
  - maybe use cgi to generate houses
  - [cool house concepts](https://coolhouseconcepts.com/house-plans/3-bedroom-bungalow-house-plan/)
  - check [houseplans.com](https://www.houseplans.com/collection/design-styles)
3. Train with labels
4. Put on server
5. Make api and do some demoing with test frontend

### Do the stuff above




# Links to possibly intereting things

* [Darknet (fast and accurate object / feature detection)](https://github.com/AlexeyAB/darknet)
* [Tutorial for Darknet implementation](https://www.youtube.com/watch?v=10joRJt39Ns)
* [Darnet Tensorflow implementation)[https://github.com/wizyoung/YOLOv3_TensorFlow]


